## Which tokens affect classifier constructions the most?

Load the best model from data augmentation experiments and the test data:

In [1]:
#!g1.1
import numpy as np
import pandas as pd
import torchtext.vocab as vocab
import torch
from razdel import sentenize
import json

In [2]:
#!g1.1
from simpletransformers.t5 import T5Model, T5Args

use_cuda = True

model_args = T5Args()
model_args.special_tokens_list = ["clf", "poss", "sg", "<nums>", "<dact>", "1ps", "2ps", "3ps", "indx", "pl"]
model_args.max_seq_length = 96
model_args.train_batch_size = 30
model_args.eval_batch_size = 30
model_args.num_train_epochs = 15
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 Russian-RSL Translation"

model = T5Model("mt5", "outputs/best_model", args=model_args)  #model15_special_tokens_mix_all.pth

### Load test data

In [3]:
#!g1.1
test_data = pd.read_csv('test_data.csv', encoding='utf-8')

test_data['stem_rus'] = test_data['test_stem_rus'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
test_data['rsl'] = test_data['test_rsl'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
test_data['gram_rus'] = test_data['test_gram_rus'].apply(lambda sent: json.loads(sent.replace('\'', '\"')))

In [4]:
#!g1.1
for index, row in test_data.iterrows():
    if row['rsl'] == ['']:
        test_data.drop(index, inplace=True)

In [5]:
#!g1.1
test_data['input_text'] = test_data['stem_rus'].apply(' '.join)
test_data['target_text'] = test_data['rsl'].apply(' '.join)

In [6]:
#!g1.1
test_data.head()

,test_stem_rus,test_gram_rus,test_rsl,stem_rus,rsl,gram_rus,input_text,target_text
0,"['а', 'когда', 'дед', 'мороз', 'вернуться', 'д...","[['CONJ'], ['CONJ'], ['им', 'мн', 'S', 'муж', ...","['indx', 'дед.мороз', 'clf', 'clf.группа', 'cl...","[а, когда, дед, мороз, вернуться, домой, мален...","[indx, дед.мороз, clf, clf.группа, clf.перемес...","[[CONJ], [CONJ], [им, мн, S, муж, од], [им, мн...",а когда дед мороз вернуться домой маленький де...,indx дед.мороз clf clf.группа clf.переместить ...
1,"['а', 'мачеха', 'лишать', 'я', 'весь', 'радость']","[['CONJ'], ['им', 'S', 'од', 'жен', 'ед'], ['и...","['мама', 'второй', '3ps', 'все', 'радость', 'д...","[а, мачеха, лишать, я, весь, радость]","[мама, второй, 3ps, все, радость, давать, 1ps,...","[[CONJ], [им, S, од, жен, ед], [изъяв, V, пе, ...",а мачеха лишать я весь радость,мама второй 3ps все радость давать 1ps ноль
4,"['зачем', 'я', 'хотеть', 'изменять', 'свой', '...","[['ADVPRO'], ['им', 'ед', '1-л', 'SPRO'], ['из...","['зачем', 'желание', 'изменение', 'все4', 'при...","[зачем, я, хотеть, изменять, свой, привычка, т...","[зачем, желание, изменение, все4, привычка, тр...","[[ADVPRO], [им, ед, 1-л, SPRO], [изъяв, V, пе,...",зачем я хотеть изменять свой привычка тратить ...,зачем желание изменение все4 привычка тратить ...
5,"['оказываться', 'впервые', 'это', 'появляться'...","[['изъяв', 'V', '3-л', 'нп', 'несов', 'ед', 'н...","['америка', 'есть', 'poss', '3ps', '3ps', 'рас...","[оказываться, впервые, это, появляться, в, аме...","[америка, есть, poss, 3ps, 3ps, рассказ, 1ps, ...","[[изъяв, V, 3-л, нп, несов, ед, непрош], [ADV]...",оказываться впервые это появляться в америка,америка есть poss 3ps 3ps рассказ 1ps интересн...
6,"['я', 'вчера', 'не', 'стирать', 'белье']","[['им', 'ед', '1-л', 'SPRO'], ['ADV'], ['PART'...","['1ps', 'вчера', 'стирать', '<dact>', 'не', 'б...","[я, вчера, не, стирать, белье]","[1ps, вчера, стирать, <dact>, не, был]","[[им, ед, 1-л, SPRO], [ADV], [PART], [изъяв, V...",я вчера не стирать белье,1ps вчера стирать <dact> не был


Try predicting something:

In [7]:
#!g1.1
text = """
    Я умею переводить текст на русский жестовый язык.
    Этот текст перевела модель машинного перевода.
    Дети играли в мяч на улице.
    Получается довольно-таки глупо
    """
to_rsl = [_.text for _ in list(sentenize(text))]
to_rsl

['Я умею переводить текст на русский жестовый язык.',
 'Этот текст перевела модель машинного перевода.',
 'Дети играли в мяч на улице.',
 'Получается довольно-таки глупо']

In [8]:
#!g1.1
rsl_preds = model.predict(to_rsl)

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/home/jupyter/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3538: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
#!g1.1
rsl_preds

['я знать переводить текст на русский жестовый язык',
 'перевести текст перевод модель этот машинный перевод',
 'ребенок играть мяч в улица на',
 'получать2 <dact> глупо']

### Preparing data

- selecting sentences that have clf in target
- saving those input texts and target texts in a separate dataset
- droping subjects -> checking number of classifiers
- droping objects -> checking number of classifiers

If I drop verbs, for sure classifiers will disappear, so it does not make any sense. 

Then I need to separately drop figures and grounds, but how do I annotate them? I save my separate dataset as csv, then open it in google docs and introduce two new columns figure ground, where I put respective noun phrases. Then I save it as a csv again

In [10]:
#!g1.1
for index, row in test_data.iterrows():
    if 'clf' not in row['target_text']:  # 112 sentences WITH *clf*
        test_data.drop(index, inplace=True)
#print(row['target_text'])

In [11]:
#!g1.1
test_data.drop(['test_stem_rus', 'test_gram_rus', 'test_rsl', 'stem_rus', 'rsl', 'gram_rus'], axis=1, inplace=True)  

In [12]:
#!g1.1
test_data.head()

,input_text,target_text
0,а когда дед мороз вернуться домой маленький де...,indx дед.мороз clf clf.группа clf.переместить ...
12,я весь устраивать я быть интересно учиться и с...,1ps спокойный2 1ps был интересный учиться и вр...
29,он долго плыть и замечать вдали берег,долго clf clf.плыть clf.на clf.бочка замечать ...
44,когда мы отжимать волос вода на они оставаться,н-о 1ps жать вода clf clf.вода clf.на clf.воло...
53,я тоже хотеть ходить на бал,1ps тоже желание посещать pl clf clf.пышная cl...


In [30]:
#!g1.1
#test_data.to_csv('figure_ground_test_set.csv', index=False)

In [13]:
#!g1.1
figure_ground_data = pd.read_csv('figure_ground_test_set.csv', encoding='utf-8')

In [16]:
#!g1.1
figure_ground_data.head(8)

,input_text,target_text,figure,ground,russian_verb
0,а когда дед мороз вернуться домой маленький де...,indx дед.мороз clf clf.группа clf.переместить ...,"дед мороз, подарок",NaN,"вернуться, раздавать"
1,я весь устраивать я быть интересно учиться и с...,1ps спокойный2 1ps был интересный учиться и вр...,NaN,NaN,скоро
2,он долго плыть и замечать вдали берег,долго clf clf.плыть clf.на clf.бочка замечать ...,он,берег,плыть
3,когда мы отжимать волос вода на они оставаться,н-о 1ps жать вода clf clf.вода clf.на clf.воло...,вода,волос,NaN
4,я тоже хотеть ходить на бал,1ps тоже желание посещать pl clf clf.пышная cl...,NaN,NaN,NaN
5,мы знать это животное по мультфильм оно выгляд...,знать вы животное2 мультфильм2 интересный clf ...,NaN,NaN,мило
6,в год в павловск построить ленинградский восст...,смотреть <nums> год <nums> год павловск <dact>...,NaN,NaN,построить
7,для кто-то нужно утяжеление для кто-то чтобы б...,<dact> кто тяжелый clf clf.волосы clf.тяжелые ...,NaN,NaN,"утяжеление, разглаживание"


I can drop figures\grounds and in a second experiment drop some other tokens in a sentence (not verb), and compare the effect on final translation:

In [20]:
#!g1.1
import re


to_rsl_no_figures = []
to_rsl_no_grounds = []

for ind, row in  figure_ground_data.iterrows():
    
    if isinstance(row['figure'], str):
        figures = row['figure'].split(', ')
        input_text =  row['input_text']
        for figure in figures:
            input_text = re.sub(figure, '', input_text)
 
        to_rsl_no_figures.append(input_text)
    else:
        to_rsl_no_figures.append(row['input_text'])
    
    if isinstance(row['ground'], str):
        print(row['ground'])
        grounds = row['ground'].split(', ')
        input_text_grounds = row['input_text']
        for ground in grounds:
            input_text_grounds = re.sub(ground, '', input_text_grounds)
        
        to_rsl_no_grounds.append(input_text_grounds)
    else:
        to_rsl_no_grounds.append(row['input_text'])

берег
волос
канализация
грудь, стол
льдина
высокий трава
кран
берег, вода
еда
шип
пройти
плечо
вы
поезд
я
капот
древолаз


In [ ]:
#!g1.1
print(len(to_rsl_no_grounds), len(to_rsl_no_figures))

That is a very small amount of data unfortunately, but I cannot take the whole training dataset, becausethe predictions would be unnaturally better, since the model has already seen those sentences.

17 with ground, 77 with figure

Make the predicitions:

In [21]:
#!g1.1
# TODO: Save results to csv and have a look at them
rsl_preds_no_figures = model.predict(to_rsl_no_figures)

Generating outputs:   0%|          | 0/4 [00:00<?, ?it/s]

/home/jupyter/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3538: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and the tokenizer under the `as_target_tokenizer` context manager to prepare
your targets.

Here is a short example:

model_inputs = tokenizer(src_texts, ...)
with tokenizer.as_target_tokenizer():
    labels = tokenizer(tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/112 [00:00<?, ?it/s]

In [22]:
#!g1.1
# TODO: Save results to csv and have a look at them
rsl_preds_no_grounds = model.predict(to_rsl_no_grounds)

Generating outputs:   0%|          | 0/4 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/112 [00:00<?, ?it/s]

In [23]:
#!g1.1
to_rsl_no_grounds[:5]

['а когда дед мороз вернуться домой маленький дед мороз приходить к главный и сказать а я то вон в лес тоже подарок раздавать',
 'я весь устраивать я быть интересно учиться и скоро я уже сам начало выполнять даже сложный заказ',
 'он долго плыть и замечать вдали ',
 'когда мы отжимать  вода на они оставаться',
 'я тоже хотеть ходить на бал']

In [24]:
#!g1.1
rsl_preds_no_grounds[:5]

['indx дед.мороз маленький домой маленький встретить приходить 1ps дед.',
 'все 1ps все устраивать 1ps интересный учиться и скоро выполнять уже готовый',
 'плакать clf clf.на clf.бочка замечать видеть',
 'н-о 1ps жать вода вода оставаться',
 '1ps тоже желание ходить 1ps бал']

In [25]:
#!g1.1
figure_ground_data['rsl_preds_no_grounds'] = rsl_preds_no_grounds
figure_ground_data['rsl_preds_no_figures'] = rsl_preds_no_figures

In [26]:
#!g1.1
truth_rsl = test_data["target_text"].tolist()

Count number of classifiers after dropping figures:

In [27]:
#!g1.1
#!g1.1
import logging
from transformers import T5ForConditionalGeneration, T5Tokenizer

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

tokenizer = T5Tokenizer.from_pretrained("cointegrated/rut5-small")
tokenizer.add_special_tokens({'additional_special_tokens': ["clf", "poss", "sg", "<nums>", "<dact>", "1ps", "2ps", "3ps", "indx", "pl"]})

Downloading:   0%|          | 0.00/625k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

9

In [28]:
#!g1.1
def accuracy_by_num_of_clf(truth_rsl, rsl_preds):
    test_length = len(truth_rsl)
    i = 0
    for truth, pred in zip(truth_rsl, rsl_preds):
        truth_tokens = [
            tokenizer.decode(input_id)
            for input_id in tokenizer(truth)['input_ids']
        ]
        pred_tokens = [
            tokenizer.decode(input_id)
            for input_id in tokenizer(pred)['input_ids']
        ]

        truth_clf_n = truth_tokens.count('clf')
        pred_clf_n = pred_tokens.count('clf')
        if truth_clf_n == pred_clf_n:
            i += 1

    acc_num = i / test_length
    print(round(acc_num*100, 3), '%')

In [29]:
#!g1.1
accuracy_by_num_of_clf(truth_rsl, rsl_preds_no_grounds)

15.179 %


In [30]:
#!g1.1
accuracy_by_num_of_clf(truth_rsl, rsl_preds_no_figures)

12.5 %


Now we can drop one token in sentence that has nothing to do with figure\ground\verb labels from the data and compare the metric

In [36]:
#!g1.1
import random


to_rsl_random_remove = []

for ind, row in  figure_ground_data.iterrows():
    
    marked_entities = []
    
    if isinstance(row['figure'], str):
        figures = row['figure'].split(', ')
        marked_entities.extend(figures)
    if isinstance(row['ground'], str):
        grounds = row['ground'].split(', ')
        marked_entities.extend(grounds)
    if isinstance(row['russian_verb'], str):
        verbs = row['russian_verb'].split(', ')
        marked_entities.extend(verbs)
    
    input_plain = row['input_text']
    for word in marked_entities:
        input_plain = re.sub(word, '', input_plain)
       
    if ' ' in input_plain:
        input_tokens = input_plain.split()
    else:
        input_tokens = [input_plain]
    sent_len = len(input_tokens)
    #print(sent_len, input_tokens)
    index = random.randint(0, sent_len-1)
    random_word_remove = input_tokens[index]
    print(index, random_word_remove)
    
    input_plain_two = row['input_text']
    if random_word_remove != '':
        to_rsl = re.sub(random_word_remove, '', input_plain_two)
        to_rsl_random_remove.append(to_rsl)
    else: 
        print('=========')
        to_rsl_random_remove.append(row['input_text'])
        

0 а
9 уже
2 замечать
5 оставаться
2 хотеть
10 мы
0 в
1 кто-то
13 поддержка
6 чинить
2 сидеть
5 по
0 на
7 тяжелый
0 этот
10 за
3 и
4 много
23 и
2 печень
6 весь
3 сердце
7 код
9 снижать
0 там
14 и
14 с
3 немного
9 участвовать
6 в
7 кто
0 а
3 красивый
1 еще
7 пищевой
0 если
10 бежать
3 и
1 тема
3 куст
5 выглядеть
13 расщеплять
2 и
1 расти
1 значит
1 уничтожать
4 папа
24 полный
12 заалеть
9 он
1 быть
8 то
3 не
0 называться
9 на
2 начинать
14 растаивать
1 ларсить
7 защищать
0 ладно
0 
10 спать
2 окно
0 первый
1 оборудование
3 слабо
0 попадать
6 комната
0 
14 выделять
4 нужно
5 весь
11 они
5 в
11 ткань
2 платить
2 бальзам
11 психосоматический
0 на
3 один
3 давать
3 начало
3 максимальный
5 знать
6 шаг
0 мочь
0 
4 трехэтажный
0 к
1 это
1 после
11 место
7 день
8 характер
7 на
0 похоже
14 то
3 снова
8 в
11 самый
8 и
3 тот
5 и
5 погодный
3 человек
7 друг
2 замечать
4 пятно
0 она
13 представление
1 дорогой
5 в


In [33]:
#!g1.1
to_rsl_random_remove

['а когда дед мороз вернуться  маленький дед мороз приходить к главный и сказать а я то вон в лес тоже подарок раздавать',
 'я  устраивать я быть интересно учиться и скоро я уже сам начало выполнять даже сложный заказ',
 'он долго плыть и  вдали берег',
 'когда мы отжимать волос вода на  оставаться',
 'я  хотеть ходить на бал',
 'мы знать это животное по  оно выглядеть так мило что мы и не думать о то что оно опасный а оно опасный',
 'в  в павловск построить ленинградский восстановительный центр',
 'для  нужно утяжеление для  чтобы быть разглаживание',
 'я обращаться и в другой место например в вога и там я тоже оформлять письмо поддержка такой образ уже они у я  несколько',
 'а этот мужчна быть сантехнк который чнть прорыв каналзаця  который облвать дерьмо тома давать он халат муж  разрешать подождать пока он првозть чстый вещь',
 'пока спасатель ехать мальчк сдеть рядом  следть',
 'он мочь бить себя по грудь или по стол пытаться  что он плохо и нужный помощь',
 'медвежонок плыть  льд

In [34]:
#!g1.1
print(len(truth_rsl), len(to_rsl_random_remove))

112 112


In [35]:
#!g1.1
accuracy_by_num_of_clf(truth_rsl, to_rsl_random_remove)

0.0 %


Why this score is so low if are careful about not dropping parts of a classifier? Maybe because we did drop mostly verbs any way, and we should cut out verbs too